In [43]:
import pandas as pd
from openai import OpenAI
import random
import os
import re

In [1952]:
category = "figurative"
idiom = "右から左"
model ="gpt-4-turbo"

client = OpenAI(
    # defaults to os.environ.get("OPENAI_API_KEY")
    api_key="OpenAI Key",
)

In [1953]:
if not os.path.exists('response_data.csv'):
    pd.DataFrame(columns=["raw_response","category", "idiom", "type"]).to_csv('response_data.csv', index=False)


In [1954]:
df_structured = pd.DataFrame(columns=["submission", "category", "idiom", "type"])

**Zero-Shot**

In [1955]:
class ChatSession:
    def __init__(self, category, model=model):
        self.model = model
        system_message = f"日本語が母語であるため、言語を豊かで創造的な方法で使うべきです。同じような構造の文を繰り返さないようにし、慣用句を適切かつ比喩的な意味で使用して文を作成してください。人名を使用して文を作成しないでください！"
        print(system_message)
        self.messages = [
            {"role": "system", "content": system_message}
        ]

    def get_completion(self, user_prompt):
        # Add the new user message
        self.messages.append({"role": "user", "content": user_prompt})
        
        # Make the API call
        response = client.chat.completions.create(
            model=self.model,
            messages=self.messages,
        )

        # Retrieve the response and append to messages to maintain context
        ai_response = response.choices[0].message.content.strip()
        self.messages.append({"role": "assistant", "content": ai_response})
        response_df = pd.DataFrame(columns=["raw_response", "idiom","type","category"])
        new_entry = {
            "raw_response": ai_response,
            "idiom": idiom,
            "type": type,
            "category":category
        }
        response_df = pd.concat([response_df, pd.DataFrame([new_entry])], ignore_index=True)
        response_df.to_csv('response_data.csv', mode='a', header=False, index=False)
        return ai_response

In [1956]:
chat_session = ChatSession(category=category)

日本語が母語であるため、言語を豊かで創造的な方法で使うべきです。同じような構造の文を繰り返さないようにし、慣用句を適切かつ比喩的な意味で使用して文を作成してください。人名を使用して文を作成しないでください！


In [1957]:
type = "Zero-shot"
step = 12

In [1958]:
prompt = f"""
"{idiom}" は日本語の慣用句です。この慣用句は、文字通りの意味でも比喩的な意味でも使用できます。比喩的な意味でこの慣用句を使って、例文を5つ作成してください。返答には文だけを書いてください。
"""
print(prompt)
print("\n")
responses = []
for i in range(step):
    result = chat_session.get_completion(prompt)
    responses.append(result)


"右から左" は日本語の慣用句です。この慣用句は、文字通りの意味でも比喩的な意味でも使用できます。比喩的な意味でこの慣用句を使って、例文を5つ作成してください。返答には文だけを書いてください。





In [1959]:
responses

['1. 彼の言ったことがいつも右から左へと流れてしまい、全然頭に残らない。\n2. 上司の指示が右から左に抜けていってしまい、結局何をすべきか分からなくなった。\n3. 会議での批判的な意見は、彼にとってはただ右から左へと通過するだけだろう。\n4. あの人は重要なメモもすぐ右から左にしてしまうから、いつも情報が抜けている。\n5. 複雑な説明を聞いても、彼女にとって右から左で、すぐに忘れてしまう。',
 '1. 私の忠告も彼には右から左へと聞き流されてしまい、全く効果がなかった。\n2. 彼女は批判を右から左に受け流して、何事も動じない様子だった。\n3. セミナーで得た情報が右から左へ抜けていき、何も手元に残らなかった。\n4. 彼はアドバイスを右から左にするタイプで、自分の頭で考えることを重んじる。\n5. 注意されても彼はいつも右から左へと流すのみで、行動を改める気配がない。',
 '1. 彼は叱られてもその注意が右から左へと耳を通り過ぎるだけで、いつも何も改めない。\n2. 会議での多くの意見が右から左に流れ、何の解決にも至らなかった。\n3. 講師の話が難しくて、学生たちは情報を右から左へと流してしまい、理解できなかった。\n4. あの人は重要なアドバイスもすぐに右から左に払いのけてしまうので、進歩が見られない。\n5. 彼女は日々の忠告を右から左にするため、同じミスを繰り返すことが多い。',
 '1. 彼は注意されるたびに言葉を右から左に押し流して、心に留めることがない。\n2. どれだけ重要な話でも、彼女にはすぐに右から左へと消えてしまう。\n3. 私のアドバイスも彼の耳には右から左へ通り過ぎる風だった。\n4. 母の言葉を右から左へと受け流す癖が、彼女の成長を妨げている気がする。\n5. 彼は受けた批判をすぐに右から左にして、何も心に残さないようだ。',
 '1. 上司の注意も彼には右から左へと通り過ぎるだけで、何の反省も見られなかった。\n2. 新しい指示が出たが、彼女はそれをすぐに右から左に払いのけ、元の方法で作業を続けた。\n3. 重要な案件の説明をしても、その内容がすぐに右から左へと抜け落ちてしまう。\n4. 彼は頻繁に受ける批評をすべて右から左へと流すため、改善の余地が見られない。\n5. 彼女は反省を促す言葉も右から左に受け流し、成長

In [1960]:
def parse_and_structure_data(raw_responses_list, idiom, category, response_type):
    # Initialize the list to store structured data
    structured_data = []
    # Iterate through each block of responses
    for block in raw_responses_list:
        # Split each block into individual sentences
        sentences = block.split('\n')
        # Remove the numbering from each sentence and strip leading/trailing whitespace
        clean_sentences = [sentence.split('. ', 1)[-1].strip() for sentence in sentences]
        # For each sentence, append a tuple with the structured data
        for sentence in clean_sentences:
            structured_data.append((sentence, category, idiom, response_type))
    return structured_data

In [1961]:
structured_data = parse_and_structure_data(responses, idiom, category, type)

new_df= pd.DataFrame(structured_data, columns=["submission", "category", "idiom", "type"])
new_df

,submission,category,idiom,type
0,彼の言ったことがいつも右から左へと流れてしまい、全然頭に残らない。,figurative,右から左,Zero-shot
1,上司の指示が右から左に抜けていってしまい、結局何をすべきか分からなくなった。,figurative,右から左,Zero-shot
2,会議での批判的な意見は、彼にとってはただ右から左へと通過するだけだろう。,figurative,右から左,Zero-shot
3,あの人は重要なメモもすぐ右から左にしてしまうから、いつも情報が抜けている。,figurative,右から左,Zero-shot
4,複雑な説明を聞いても、彼女にとって右から左で、すぐに忘れてしまう。,figurative,右から左,Zero-shot
5,私の忠告も彼には右から左へと聞き流されてしまい、全く効果がなかった。,figurative,右から左,Zero-shot
6,彼女は批判を右から左に受け流して、何事も動じない様子だった。,figurative,右から左,Zero-shot
7,セミナーで得た情報が右から左へ抜けていき、何も手元に残らなかった。,figurative,右から左,Zero-shot
8,彼はアドバイスを右から左にするタイプで、自分の頭で考えることを重んじる。,figurative,右から左,Zero-shot
9,注意されても彼はいつも右から左へと流すのみで、行動を改める気配がない。,figurative,右から左,Zero-shot


In [1962]:
new_df = new_df[new_df['submission']!=""]
new_df.reset_index(drop=True,inplace=True)
new_df


,submission,category,idiom,type
0,彼の言ったことがいつも右から左へと流れてしまい、全然頭に残らない。,figurative,右から左,Zero-shot
1,上司の指示が右から左に抜けていってしまい、結局何をすべきか分からなくなった。,figurative,右から左,Zero-shot
2,会議での批判的な意見は、彼にとってはただ右から左へと通過するだけだろう。,figurative,右から左,Zero-shot
3,あの人は重要なメモもすぐ右から左にしてしまうから、いつも情報が抜けている。,figurative,右から左,Zero-shot
4,複雑な説明を聞いても、彼女にとって右から左で、すぐに忘れてしまう。,figurative,右から左,Zero-shot
5,私の忠告も彼には右から左へと聞き流されてしまい、全く効果がなかった。,figurative,右から左,Zero-shot
6,彼女は批判を右から左に受け流して、何事も動じない様子だった。,figurative,右から左,Zero-shot
7,セミナーで得た情報が右から左へ抜けていき、何も手元に残らなかった。,figurative,右から左,Zero-shot
8,彼はアドバイスを右から左にするタイプで、自分の頭で考えることを重んじる。,figurative,右から左,Zero-shot
9,注意されても彼はいつも右から左へと流すのみで、行動を改める気配がない。,figurative,右から左,Zero-shot


In [1963]:
df_structured = pd.concat([df_structured, new_df], ignore_index=True)
df_structured

,submission,category,idiom,type
0,彼の言ったことがいつも右から左へと流れてしまい、全然頭に残らない。,figurative,右から左,Zero-shot
1,上司の指示が右から左に抜けていってしまい、結局何をすべきか分からなくなった。,figurative,右から左,Zero-shot
2,会議での批判的な意見は、彼にとってはただ右から左へと通過するだけだろう。,figurative,右から左,Zero-shot
3,あの人は重要なメモもすぐ右から左にしてしまうから、いつも情報が抜けている。,figurative,右から左,Zero-shot
4,複雑な説明を聞いても、彼女にとって右から左で、すぐに忘れてしまう。,figurative,右から左,Zero-shot
5,私の忠告も彼には右から左へと聞き流されてしまい、全く効果がなかった。,figurative,右から左,Zero-shot
6,彼女は批判を右から左に受け流して、何事も動じない様子だった。,figurative,右から左,Zero-shot
7,セミナーで得た情報が右から左へ抜けていき、何も手元に残らなかった。,figurative,右から左,Zero-shot
8,彼はアドバイスを右から左にするタイプで、自分の頭で考えることを重んじる。,figurative,右から左,Zero-shot
9,注意されても彼はいつも右から左へと流すのみで、行動を改める気配がない。,figurative,右から左,Zero-shot


**Enhanced-prompting**

In [1964]:
type = "Enhanced-prompting"
step = 1

In [1965]:
class ChatSession:
    def __init__(self, category, model=model):
        self.model = model
        system_message = f"日本語が母語であるため、言語を豊かで創造的に使うようにしてください。求められた要望に従って回答するように心がけてください。慣用句の語根を変更せずに使用し、人名を用いて文を作成しないでください！"
        print(system_message)
        self.messages = [
            {"role": "system", "content": system_message}
        ]

    def get_completion(self, user_prompt):
        # Add the new user message
        self.messages.append({"role": "user", "content": user_prompt})
        
        # Make the API call
        response = client.chat.completions.create(
            model=self.model,
            messages=self.messages,
        )

        # Retrieve the response and append to messages to maintain context
        ai_response = response.choices[0].message.content.strip()
        self.messages.append({"role": "assistant", "content": ai_response})
        response_df = pd.DataFrame(columns=["raw_response", "idiom","type","category"])
        new_entry = {
            "raw_response": ai_response,
            "idiom": idiom,
            "type": type,
            "category":category
        }
        response_df = pd.concat([response_df, pd.DataFrame([new_entry])], ignore_index=True)
        response_df.to_csv('response_data.csv', mode='a', header=False, index=False)
        return ai_response

prompt = f"""
"猫に小判" は、価値を理解できない人に貴重なものを与えたとき、努力が無駄になったとき、または相手が提供されたチャンスを活かせなかったときに使用される日本語の慣用句です。同様に、"{idiom}" という慣用句が比喩的に使用される状況をリストアップしていただけますか？
"""



In [1966]:
chat_session = ChatSession(category=category)

日本語が母語であるため、言語を豊かで創造的に使うようにしてください。求められた要望に従って回答するように心がけてください。慣用句の語根を変更せずに使用し、人名を用いて文を作成しないでください！


In [1967]:
print(prompt)
print("\n")

for i in range(step):
    result = chat_session.get_completion(prompt)
    print(result)


"猫に小判" は、価値を理解できない人に貴重なものを与えたとき、努力が無駄になったとき、または相手が提供されたチャンスを活かせなかったときに使用される日本語の慣用句です。同様に、"右から左" という慣用句が比喩的に使用される状況をリストアップしていただけますか？



もちろんです。"右から左" という慣用句は、何かを真剣に受け止めず、注意を払わない状況や、取り扱いが杜撰である様子を表現する際に使用されることがあります。以下に、その慣用句が使われる具体的な状況をリストアップします：

1. **指示を無視する場合**：
   - 上司からの重要な指示があったにもかかわらず、それをすぐに忘れたり無視したりする場合。

2. **情報をすぐに忘れる場合**：
   - 会議で話された内容を聞いているのに、すぐにその情報を忘れてしまうとき。

3. **お金の取り扱いが軽率な場合**：
   - 収入があったとしてもすぐに使ってしまい、貯金などにまわさない場合。

4. **情報を他人に伝える際の軽率さ**：
   - 大切なメッセージを他人から受け取ったときに、その重要性を理解せずサッと他の人に伝えてしまう場合。

5. **学びを生かさない場合**：
   - 新しい知識や技術を学んだはずだが、実際の行動や仕事には活かされていない時。

これらの状況は、"右から左" が表す、「何も留まらずにそのまま通り過ぎる」という意味合いを色濃く反映しています。要するに、入ってきた情報や物事をしっかりと捉えず、すぐに他へ流してしまう様子を指しています。


In [1968]:
prompt = f"""
"{idiom}" という慣用句が前のメッセージでリストされた各状況に比喩的に使われる4つの異なる文を作成してください。最初の文は長くて詳細で、二番目の文は疑問形、三番目の文は否定形、四番目の文は慣用句を構成する単語の間に他の単語が入る形でなければなりません。この慣用句の語根を変えずに。各カテゴリーについて4つの異なる文を作成してください。返答としてこれらの文だけを返してください。語根を変更せずに。
"""

In [1969]:
print(prompt)
responses = []
for i in range(step):
    result = chat_session.get_completion(prompt)
    responses.append(result)


"右から左" という慣用句が前のメッセージでリストされた各状況に比喩的に使われる4つの異なる文を作成してください。最初の文は長くて詳細で、二番目の文は疑問形、三番目の文は否定形、四番目の文は慣用句を構成する単語の間に他の単語が入る形でなければなりません。この慣用句の語根を変えずに。各カテゴリーについて4つの異なる文を作成してください。返答としてこれらの文だけを返してください。語根を変更せずに。



In [1970]:
responses

['1. 長くて詳細な文：\n   - 昨日行われた重要な会議で課長が新しいプロジェクトについて詳細な説明をしたにもかかわらず、彼はその情報を右から左へと流してしまい、今日のミーティングでは全く手掛かりがない状態だった。\n   - 部長からの厳重な注意を受けたはずなのに、彼女はその言葉を右から左へと流して、また同じ過ちを犯してしまった。\n   - 彼は今朝、受け取った給料を右から左へと流してしまい、もう手元には何も残っていない。\n   - 彼女は他人からの有益な助言をいつも右から左へと流してしまうため、成長の機会を常に逃してしまいます。\n\n2. 疑問形の文：\n   - なぜ彼はいつも重要な情報を右から左へと扱ってしまうのでしょうか？\n   - 彼女はなぜその重要なメッセージを右から左へと伝えただけなの？\n   - 彼はどうしてそのお金を右から左へと流すのですか？\n   - あのアドバイスを彼女はなぜ右から左へと聞き流すのでしょう？\n\n3. 否定形の文：\n   - 彼は右から左へと扱うことなく、今回のフィードバックを真剣に受け止めた。\n   - 彼女は重要な指示を右から左へと流さなかった。\n   - 今月は給料を右から左へと使わなかった。\n   - 彼女は提案された解決策を右から左へと聞き流さなかった。\n\n4. 慣用句の構成単語の間に他の単語が入る形：\n   - 彼は情報を右の手から左の手へと無造作に扱い、重要な点を見落としてしまった。\n   - なんとなく受けたフィードバックが右の耳から左の耳へと抜けてしまい、何も身についていない。\n   - 彼は昨日のボーナスを右のポケットから左のポケットへと移すだけで、何の計画もなしに使い果たしてしまった。\n   - 彼女はチームからのアイディアを右の目から左の目へと一瞥するだけで、深く考えることなく次へ進んでしまった。']

In [1971]:
def parse_and_structure_data(raw_responses_list, idiom, category, response_type):
    structured_data = []
    # Iterate through each block of responses
    for block in raw_responses_list:
        # Split each block into individual lines
        lines = block.split('\n')
        # Skip the header line (assumes the header is the first line)
        for line in lines[1:]:  # this skips the header which is lines[0]
            # Check if the line contains an actual sentence
            if line.strip() and '-' in line:
                # Remove the bullet point and any leading/trailing spaces
                sentence = line.split('-', 1)[-1].strip()
                # Append the structured tuple
                structured_data.append((sentence, category, idiom, response_type))
    return structured_data

In [1972]:
def parse_numbered_sentences(raw_texts):
    # Initialize the list to store extracted sentences
    extracted_sentences = []
    
    # Iterate through each text block in the list
    for raw_text in raw_texts:
        # Split the input text into lines
        lines = raw_text.split('\n')
        
        # Iterate through each line in the input text
        for line in lines:
            # Strip any leading/trailing whitespace from the line
            line = line.strip()
            
            # Check if the line starts with a numeric prefix followed by a dot and space (e.g., '1. ')
            if line.startswith(('1. ', '2. ', '3. ', '4. ')):
                # Extract the sentence after the numeric prefix
                sentence = line.split('. ', 1)[-1]
                # Append the extracted sentence to the list
                extracted_sentences.append((sentence, category, idiom, type))
    
    return extracted_sentences

In [1973]:
parsed_data = parse_and_structure_data(responses, idiom, category, type)
parsed_data

[('昨日行われた重要な会議で課長が新しいプロジェクトについて詳細な説明をしたにもかかわらず、彼はその情報を右から左へと流してしまい、今日のミーティングでは全く手掛かりがない状態だった。',
  'figurative',
  '右から左',
  'Enhanced-prompting'),
 ('部長からの厳重な注意を受けたはずなのに、彼女はその言葉を右から左へと流して、また同じ過ちを犯してしまった。',
  'figurative',
  '右から左',
  'Enhanced-prompting'),
 ('彼は今朝、受け取った給料を右から左へと流してしまい、もう手元には何も残っていない。',
  'figurative',
  '右から左',
  'Enhanced-prompting'),
 ('彼女は他人からの有益な助言をいつも右から左へと流してしまうため、成長の機会を常に逃してしまいます。',
  'figurative',
  '右から左',
  'Enhanced-prompting'),
 ('なぜ彼はいつも重要な情報を右から左へと扱ってしまうのでしょうか？',
  'figurative',
  '右から左',
  'Enhanced-prompting'),
 ('彼女はなぜその重要なメッセージを右から左へと伝えただけなの？',
  'figurative',
  '右から左',
  'Enhanced-prompting'),
 ('彼はどうしてそのお金を右から左へと流すのですか？', 'figurative', '右から左', 'Enhanced-prompting'),
 ('あのアドバイスを彼女はなぜ右から左へと聞き流すのでしょう？', 'figurative', '右から左', 'Enhanced-prompting'),
 ('彼は右から左へと扱うことなく、今回のフィードバックを真剣に受け止めた。',
  'figurative',
  '右から左',
  'Enhanced-prompting'),
 ('彼女は重要な指示を右から左へと流さなかった。', 'figurative', '右から左', 'Enhanced-prompting'),
 ('今月は給料を右から左へと使わなかった。', 'figurative', '右から

In [1974]:
new_df= pd.DataFrame(parsed_data, columns=["submission", "category", "idiom", "type"])
new_df

,submission,category,idiom,type
0,昨日行われた重要な会議で課長が新しいプロジェクトについて詳細な説明をしたにもかかわらず、彼は...,figurative,右から左,Enhanced-prompting
1,部長からの厳重な注意を受けたはずなのに、彼女はその言葉を右から左へと流して、また同じ過ちを犯...,figurative,右から左,Enhanced-prompting
2,彼は今朝、受け取った給料を右から左へと流してしまい、もう手元には何も残っていない。,figurative,右から左,Enhanced-prompting
3,彼女は他人からの有益な助言をいつも右から左へと流してしまうため、成長の機会を常に逃してしまいます。,figurative,右から左,Enhanced-prompting
4,なぜ彼はいつも重要な情報を右から左へと扱ってしまうのでしょうか？,figurative,右から左,Enhanced-prompting
5,彼女はなぜその重要なメッセージを右から左へと伝えただけなの？,figurative,右から左,Enhanced-prompting
6,彼はどうしてそのお金を右から左へと流すのですか？,figurative,右から左,Enhanced-prompting
7,あのアドバイスを彼女はなぜ右から左へと聞き流すのでしょう？,figurative,右から左,Enhanced-prompting
8,彼は右から左へと扱うことなく、今回のフィードバックを真剣に受け止めた。,figurative,右から左,Enhanced-prompting
9,彼女は重要な指示を右から左へと流さなかった。,figurative,右から左,Enhanced-prompting


In [1975]:
df_structured = pd.concat([df_structured, new_df], ignore_index=True)
df_structured

,submission,category,idiom,type
0,彼の言ったことがいつも右から左へと流れてしまい、全然頭に残らない。,figurative,右から左,Zero-shot
1,上司の指示が右から左に抜けていってしまい、結局何をすべきか分からなくなった。,figurative,右から左,Zero-shot
2,会議での批判的な意見は、彼にとってはただ右から左へと通過するだけだろう。,figurative,右から左,Zero-shot
3,あの人は重要なメモもすぐ右から左にしてしまうから、いつも情報が抜けている。,figurative,右から左,Zero-shot
4,複雑な説明を聞いても、彼女にとって右から左で、すぐに忘れてしまう。,figurative,右から左,Zero-shot
...,...,...,...,...
71,彼女は提案された解決策を右から左へと聞き流さなかった。,figurative,右から左,Enhanced-prompting
72,彼は情報を右の手から左の手へと無造作に扱い、重要な点を見落としてしまった。,figurative,右から左,Enhanced-prompting
73,なんとなく受けたフィードバックが右の耳から左の耳へと抜けてしまい、何も身についていない。,figurative,右から左,Enhanced-prompting
74,彼は昨日のボーナスを右のポケットから左のポケットへと移すだけで、何の計画もなしに使い果たして...,figurative,右から左,Enhanced-prompting


In [1976]:
class ChatSession:
    def __init__(self, category, model=model):
        self.model = model
        system_message = f"日本語が母語であるため、言語を豊かで創造的な方法で使うべきです。同じ文を繰り返さないようにし、慣用句を正しく、そして比喩的な意味で使って文を作成してください。慣用句の語根を変えずに。人名を使って文を作成しないでください！"
        print(system_message)
        self.messages = [
            {"role": "system", "content": system_message}
        ]

    def get_completion(self, user_prompt):
        # Add the new user message
        self.messages.append({"role": "user", "content": user_prompt})
        
        # Make the API call
        response = client.chat.completions.create(
            model=self.model,
            messages=self.messages,
        )

        # Retrieve the response and append to messages to maintain context
        ai_response = response.choices[0].message.content.strip()
        self.messages.append({"role": "assistant", "content": ai_response})
        response_df = pd.DataFrame(columns=["raw_response", "idiom","type","category"])
        new_entry = {
            "raw_response": ai_response,
            "idiom": idiom,
            "type": type,
            "category":category
        }
        response_df = pd.concat([response_df, pd.DataFrame([new_entry])], ignore_index=True)
        response_df.to_csv('response_data.csv', mode='a', header=False, index=False)
        return ai_response

In [1977]:
chat_session = ChatSession(category=category)

日本語が母語であるため、言語を豊かで創造的な方法で使うべきです。同じ文を繰り返さないようにし、慣用句を正しく、そして比喩的な意味で使って文を作成してください。慣用句の語根を変えずに。人名を使って文を作成しないでください！


In [1983]:
prompt = f"""
"{idiom}" という慣用句を比喩的な意味で使用し、異なる文構造で5つの新しい文を作成してください。文の異なる部分で慣用句を使用してください。同じ文を繰り返さないようにしてください。
"""
print(prompt)
print("\n")
responses = []
for i in range(4):
    result = chat_session.get_completion(prompt)
    responses.append(result)


"右から左" という慣用句を比喩的な意味で使用し、異なる文構造で5つの新しい文を作成してください。文の異なる部分で慣用句を使用してください。同じ文を繰り返さないようにしてください。





In [1984]:
responses

['1. 彼は注意を聞いても、すぐに“右から左”に流すため、何度も同じミスを繰り返す。\n2. 重要なメールが届いたが、彼女はそれを右から左に流してしまい、重要な期限を逃した。\n3. 社長の指示も、あのチームでは右から左に行ってしまうことが多く、改善が見られない。\n4. ミーティングでのフィードバックは、彼にとって右から左に流れるもので、影響を受けなかった。\n5. 彼女はアドバイスを右から左に耳を通すだけで、実際には何も行動に移さない。',
 '1. 隣人からのクレームは常に彼にとっては右から左だった; 彼は決して真剣に取り合わなかった。\n2. 彼女は悪意のある噂を右から左に扱い、自分の幸福に集中し続けた。\n3. 会議室での提言が右から左へと飛んでいき、誰の耳にも留まることがなかった。\n4. 彼は与えられた批評をただ右から左に捉えるだけで、改善に繋がることが少ない。\n5. 改善案が彼の前を右から左へと通り過ぎても、彼はいつも通りのやり方を変えようとはしなかった。',
 '1. 新しい安全規則が導入されたが、多くの従業員はそれを右から左に聞き流して、変わらない日常を送っている。\n2. 彼女の忠告がいつも右から左に通過するので、友人はまた同じ過ちを犯してしまった。\n3. 老人は若者の話を右から左に受け流し、その経験の価値を理解しなかった。\n4. 彼はイベントでの注意事項を右から左に流す傾向があり、しばしば重要な情報を見落としてしまう。\n5. 議論中、彼の意見はあまりにも速く右から左に受け流され、まるで空気のように無視された。',
 '1. 彼は会議の間、重要な指摘を右から左に取り扱い、その後のプロジェクトに悪影響を及ぼした。\n2. 告知されたルール変更は、チームにとってあまりに些細で、皆の意識を右から左に過ぎ去った。\n3. ビジネスメールが彼女のインボックスを埋め尽くすも、多くは右から左にスクロールされて未読のままだ。\n4. 上司からの警告は彼にとってただの風となり、どれもが右から左に押し流されてしまう。\n5. 講師のアドバイスが右から左に消えていく中、学生たちはそれぞれの理解のペースで学習を続けていた。']

In [1985]:
def parse_and_structure_data(raw_responses_list, idiom, category, response_type):
    # Initialize the list to store structured data
    structured_data = []
    # Iterate through each block of responses
    for block in raw_responses_list:
        # Split each block into individual sentences
        sentences = block.split('\n')
        # Remove the numbering from each sentence and strip leading/trailing whitespace
        clean_sentences = [sentence.split('. ', 1)[-1].strip() for sentence in sentences]
        # For each sentence, append a tuple with the structured data
        for sentence in clean_sentences:
            structured_data.append((sentence, category, idiom, response_type))
    return structured_data

In [1986]:
structured_data = parse_and_structure_data(responses, idiom, category, type)

new_df= pd.DataFrame(structured_data, columns=["submission", "category", "idiom", "type"])
new_df

,submission,category,idiom,type
0,彼は注意を聞いても、すぐに“右から左”に流すため、何度も同じミスを繰り返す。,figurative,右から左,Enhanced-prompting
1,重要なメールが届いたが、彼女はそれを右から左に流してしまい、重要な期限を逃した。,figurative,右から左,Enhanced-prompting
2,社長の指示も、あのチームでは右から左に行ってしまうことが多く、改善が見られない。,figurative,右から左,Enhanced-prompting
3,ミーティングでのフィードバックは、彼にとって右から左に流れるもので、影響を受けなかった。,figurative,右から左,Enhanced-prompting
4,彼女はアドバイスを右から左に耳を通すだけで、実際には何も行動に移さない。,figurative,右から左,Enhanced-prompting
5,隣人からのクレームは常に彼にとっては右から左だった; 彼は決して真剣に取り合わなかった。,figurative,右から左,Enhanced-prompting
6,彼女は悪意のある噂を右から左に扱い、自分の幸福に集中し続けた。,figurative,右から左,Enhanced-prompting
7,会議室での提言が右から左へと飛んでいき、誰の耳にも留まることがなかった。,figurative,右から左,Enhanced-prompting
8,彼は与えられた批評をただ右から左に捉えるだけで、改善に繋がることが少ない。,figurative,右から左,Enhanced-prompting
9,改善案が彼の前を右から左へと通り過ぎても、彼はいつも通りのやり方を変えようとはしなかった。,figurative,右から左,Enhanced-prompting


In [1987]:
df_structured = pd.concat([df_structured, new_df], ignore_index=True)
df_structured

,submission,category,idiom,type
0,彼の言ったことがいつも右から左へと流れてしまい、全然頭に残らない。,figurative,右から左,Zero-shot
1,上司の指示が右から左に抜けていってしまい、結局何をすべきか分からなくなった。,figurative,右から左,Zero-shot
2,会議での批判的な意見は、彼にとってはただ右から左へと通過するだけだろう。,figurative,右から左,Zero-shot
3,あの人は重要なメモもすぐ右から左にしてしまうから、いつも情報が抜けている。,figurative,右から左,Zero-shot
4,複雑な説明を聞いても、彼女にとって右から左で、すぐに忘れてしまう。,figurative,右から左,Zero-shot
...,...,...,...,...
95,彼は会議の間、重要な指摘を右から左に取り扱い、その後のプロジェクトに悪影響を及ぼした。,figurative,右から左,Enhanced-prompting
96,告知されたルール変更は、チームにとってあまりに些細で、皆の意識を右から左に過ぎ去った。,figurative,右から左,Enhanced-prompting
97,ビジネスメールが彼女のインボックスを埋め尽くすも、多くは右から左にスクロールされて未読のままだ。,figurative,右から左,Enhanced-prompting
98,上司からの警告は彼にとってただの風となり、どれもが右から左に押し流されてしまう。,figurative,右から左,Enhanced-prompting


In [1988]:
# Define the path for the CSV file
file_path = 'gpt4_jp_mecaz.csv'

# Check if the file exists
if os.path.exists(file_path):
    # Append without writing headers
    df_structured.to_csv(file_path, mode='a', index=False, header=False,encoding='euc-jp')
else:
    # Create a new file and write the DataFrame with headers
    df_structured.to_csv(file_path, mode='w', index=False, header=True,encoding='euc-jp')